In [30]:
from tkinter import *
master = Tk()
Label(master, text='Enter Review: ').grid(row=0)
e1 = Entry(master)
e1.grid(row=0, column=1)

def buttonClick():
        review = e1.get()
        print(review)
        
button = Button(master, text="Enter", command=buttonClick)
button.grid(row=3, column=1)


mainloop()